In [3]:
using DataFrames, CSV
using Pkg
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots
using Dates
using Tables

In [4]:
#A is the activity set (?)
#E is the event set

#L is lower bound on 
#U is upper bound on

In [5]:
Lines = ["HSR","CR"]

2-element Vector{String}:
 "HSR"
 "CR"

In [6]:
#i in E = line, station, eventType TUPLE
E = [1,2,3,4,5,6,7,8,9,10,11]

11-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

In [46]:
A = Array{Any}(undef, 11, 11)

for i in 1:11
    for j in 1:11
        A[i,j] = [0,20]
    end
end

A[1,2] = [30,36]
A[2,3] = [18,24]
A[4,5] = [24,28]
A[5,6] = [14,20]
A[6,7] = [20,24]
A[8,9] = [24,28]
A[9,10] = [14,20]
A[10,11] = [14,18]

A[1,4] = [3,t]
A[4,1] = [3,t]
A[1,8] = [3,t]
A[8,1] = [3,t]
A[4,8] = [3,t]
A[8,4] = [3,t]

A[2,5] = [3,t]
A[5,2] = [3,t]
A[2,9] = [3,t]
A[9,2] = [3,t]
A[5,9] = [3,t]
A[9,5] = [3,t]

A[3,6] = [3,t]
A[6,3] = [3,t]
A[3,10] = [3,t]
A[10,3] = [3,t]
A[6,10] = [3,t]
A[10,6] = [3,t]

A[7,11] = [3,t]
A[11,7] =[3,t]

2-element Vector{Int64}:
   3
 120

In [47]:
A

11×11 Matrix{Any}:
 [0, 20]   [30, 36]  [0, 20]   [3, 120]  …  [0, 20]   [0, 20]   [0, 20]
 [0, 20]   [0, 20]   [18, 24]  [0, 20]      [3, 120]  [0, 20]   [0, 20]
 [0, 20]   [0, 20]   [0, 20]   [0, 20]      [0, 20]   [3, 120]  [0, 20]
 [3, 120]  [0, 20]   [0, 20]   [0, 20]      [0, 20]   [0, 20]   [0, 20]
 [0, 20]   [3, 120]  [0, 20]   [0, 20]      [3, 120]  [0, 20]   [0, 20]
 [0, 20]   [0, 20]   [3, 120]  [0, 20]   …  [0, 20]   [3, 120]  [0, 20]
 [0, 20]   [0, 20]   [0, 20]   [0, 20]      [0, 20]   [0, 20]   [3, 120]
 [3, 120]  [0, 20]   [0, 20]   [3, 120]     [24, 28]  [0, 20]   [0, 20]
 [0, 20]   [3, 120]  [0, 20]   [0, 20]      [0, 20]   [14, 20]  [0, 20]
 [0, 20]   [0, 20]   [3, 120]  [0, 20]      [0, 20]   [0, 20]   [14, 18]
 [0, 20]   [0, 20]   [0, 20]   [0, 20]   …  [0, 20]   [0, 20]   [0, 20]

In [48]:
L = 8
U = 100

100

In [57]:
min(1,5)

1

In [80]:
const GRB_ENV = Gurobi.Env();
m = Model(() -> Gurobi.Optimizer(GRB_ENV));

@variable(m, t >= L); 
@variable(m, Tau[1:length(E)] >= 0, Int); #main decision variable, time of event i 
@variable(m, y[1:length(E),1:length(E)] >= 0); #linearization variable y=zt
@variable(m, z[1:length(E),1:length(E)], Bin); # describes for each activity the order of events within 
                                                ## the timetable period (1 if transverses to next cycle)



Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-19


11×11 Matrix{VariableRef}:
 z[1,1]   z[1,2]   z[1,3]   z[1,4]   z[1,5]   …  z[1,9]   z[1,10]   z[1,11]
 z[2,1]   z[2,2]   z[2,3]   z[2,4]   z[2,5]      z[2,9]   z[2,10]   z[2,11]
 z[3,1]   z[3,2]   z[3,3]   z[3,4]   z[3,5]      z[3,9]   z[3,10]   z[3,11]
 z[4,1]   z[4,2]   z[4,3]   z[4,4]   z[4,5]      z[4,9]   z[4,10]   z[4,11]
 z[5,1]   z[5,2]   z[5,3]   z[5,4]   z[5,5]      z[5,9]   z[5,10]   z[5,11]
 z[6,1]   z[6,2]   z[6,3]   z[6,4]   z[6,5]   …  z[6,9]   z[6,10]   z[6,11]
 z[7,1]   z[7,2]   z[7,3]   z[7,4]   z[7,5]      z[7,9]   z[7,10]   z[7,11]
 z[8,1]   z[8,2]   z[8,3]   z[8,4]   z[8,5]      z[8,9]   z[8,10]   z[8,11]
 z[9,1]   z[9,2]   z[9,3]   z[9,4]   z[9,5]      z[9,9]   z[9,10]   z[9,11]
 z[10,1]  z[10,2]  z[10,3]  z[10,4]  z[10,5]     z[10,9]  z[10,10]  z[10,11]
 z[11,1]  z[11,2]  z[11,3]  z[11,4]  z[11,5]  …  z[11,9]  z[11,10]  z[11,11]

In [81]:
A = Array{Any}(undef, 11, 11)

for i in 1:11
    for j in 1:11
        A[i,j] = [0,20]
    end
end

A[1,2] = [30,36]
A[2,3] = [18,24]
A[4,5] = [24,28]
A[5,6] = [14,20]
A[6,7] = [20,24]
A[8,9] = [24,28]
A[9,10] = [14,20]
A[10,11] = [14,18]

A[1,4] = [3,t-3]
A[4,1] = [3,t-3]
A[1,8] = [3,t-3]
A[8,1] = [3,t-3]
A[4,8] = [3,t-3]
A[8,4] = [3,t-3]

A[2,5] = [3,t-3]
A[5,2] = [3,t-3]
A[2,9] = [3,t-3]
A[9,2] = [3,t-3]
A[5,9] = [3,t-3]
A[9,5] = [3,t-3]

A[3,6] = [3,t-3]
A[6,3] = [3,t-3]
A[3,10] = [3,t-3]
A[10,3] = [3,t-3]
A[6,10] = [3,t-3]
A[10,6] = [3,t-3]

A[7,11] = [3,t-3]
A[11,7] =[3,t-3]

2-element Vector{AffExpr}:
 3
 t - 3

In [82]:

@objective(m, Min, t);

@constraint(m, c_1a[i in 1:length(E), j in 1:length(E)], Tau[j] - Tau[i] + y[i,j] >= A[i,j][1]); #the time between events should be
                                                                                                 ## bounded by the given bounds
@constraint(m, c_1b[i in 1:length(E), j in 1:length(E)], Tau[j] - Tau[i] + y[i,j] <= A[i,j][2]);

@constraint(m, c_2[i in 1:length(E)], Tau[i] - t <= .0001); #Start time of all events must be less than the cycle time

@constraint(m, c_3[i in 1:length(E), j in 1:length(E)], y[i,j] <= t); #linearization constraint

@constraint(m, c_4[i in 1:length(E), j in 1:length(E)], y[i,j] - U*z[i,j] <= 0); #linearization constraint

@constraint(m, c_5[i in 1:length(E), j in 1:length(E)], y[i,j] - t + U*(1- z[i,j]) >= 0); #linearization constraint

@constraint(m, c_6, t <= U); #upper bound on cycle time

In [83]:
optimize!(m)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 617 rows, 254 columns and 1572 nonzeros
Model fingerprint: 0x296dec67
Variable types: 122 continuous, 132 integer (121 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [8e+00, 8e+00]
  RHS range        [1e-04, 1e+02]
Presolve removed 23 rows and 0 columns
Presolve time: 0.01s
Presolved: 594 rows, 254 columns, 1549 nonzeros
Variable types: 122 continuous, 132 integer (121 binary)

Root relaxation: objective 1.600000e+01, 167 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   16.00000    0    2          -   16.00000      -     -    0s
H    0     0                      17.9999000   16.00000  11.1%     -    0s

In [84]:
value.(Tau)

11-element Vector{Float64}:
 -0.0
 14.0
 17.0
  3.0
 10.0
 11.0
 16.0
  6.0
 17.0
 14.0
 11.0

In [55]:
value.(y)

11×11 Matrix{Float64}:
 2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9
 2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9
 2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9
 2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9
 2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9
 2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9
 2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9
 2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9
 2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9
 2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9
 2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9  2.0e9